# SDoH Extraction using LLMs

## Setup

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os
from pathlib import Path
import sys

# Add the project root to the Python path to import the modules
project_root = Path().absolute().parent
sys.path.append(str(project_root))

## Load models

In [4]:
import torch
import transformers

# Use shared cache
os.environ['HF_HOME'] = '/data/resource/huggingface'
os.environ['TRANSFORMERS_CACHE'] = '/data/resource/huggingface/hub'
os.environ['TRANSFORMERS_OFFLINE'] = '1'  # Force offline mode

# What models are available
cache_dir = "/data/resource/huggingface/hub"
available_models = []

if os.path.exists(cache_dir):
    for item in os.listdir(cache_dir):
        if item.startswith("models--"):
            # Convert models--org--name to org/name format
            model_name = item.replace("models--", "").replace("--", "/")
            available_models.append(model_name)

print("Available cached models:")
for model in sorted(available_models):
    print(f"  {model}")


/opt/anaconda/envs/keble8263-sdoh-extraction/lib/python3.10/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Available cached models:
  CohereForAI/aya-23-35B
  CohereForAI/aya-23-8B
  CohereForAI/aya-vision-8b
  HuggingFaceTB/SmolLM-135M-Instruct
  LLaMAX/LLaMAX3-8B-Alpaca
  Qwen/Qwen2.5-1.5B
  Qwen/Qwen2.5-3B
  Qwen/Qwen2.5-72B-Instruct
  Qwen/Qwen2.5-7B
  Qwen/Qwen2.5-7B-Instruct
  Qwen/Qwen2.5-7B-instruct
  Qwen/Qwen2.5-VL-7B-Instruct
  Unbabel/wmt20-comet-qe-da
  Unbabel/wmt22-comet-da
  bert-base-uncased
  bert-large-uncased
  cardiffnlp/twitter-roberta-base-sentiment
  clairebarale/refugee_cases_ner
  deepseek-ai/DeepSeek-R1-Distill-Llama-70B
  deepseek-ai/DeepSeek-R1-Distill-Llama-8B
  deepseek-ai/DeepSeek-R1-Distill-Qwen-14B
  deepseek-ai/DeepSeek-R1-Distill-Qwen-32B
  deepseek-ai/DeepSeek-R1-Distill-Qwen-7B
  facebook/nllb-200-3.3B
  facebook/nllb-200-distilled-1.3B
  facebook/nllb-200-distilled-600M
  gpt2
  gpt2-medium
  gpt2-xl
  hfl/chinese-electra-180g-small-discriminator
  hfl/chinese-legal-electra-base-discriminator
  hfl/chinese-legal-electra-small-discriminator
  hfl/chines

In [5]:
print(f"Transformers version: {transformers.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

Transformers version: 4.52.3
PyTorch version: 2.6.0
CUDA available: True


## Prototype zero-shot extraction of SDoH from a note

In [6]:
import pandas as pd

# Load cleaned data
brc_referrals_cleaned = pd.read_csv("../data/processed/BRC_referrals_cleaned.csv")

In [7]:
# Load a specific note: Case Reference = CAS-467812
sample_note = brc_referrals_cleaned[brc_referrals_cleaned['Case Reference'] == 'CAS-467812'].iloc[0]['Referral Notes (depersonalised)']

print("Sample Note:")
print(sample_note)

Sample Note:
Befriending. Lives with husband for whom patient is carer. Living on ready meals at present. PERSON concerned that they may not be eating / drinking enough. Carers in XXXX times daily for patient to help with washing / dressing. Depending on side - effects of radiotherapy , patient may go on to PEG feeding. Patient feeling slightly overwhelmed by everything. FPOC and Carers Support Shropshire numbers given to patient ’s daughter ( with patient consent ) but would value additional emotional support / befriending / check ins to make sure they are eating / drinking etc as all activities of daily living severely compromised at present. Very supportive daughter who lives in PERSON. The patient is due to start radiotherapy on XXXX January so support during this time would be particularly beneficial. Due to start radiotherapy on XXXX at SATH EOS XXXX.


### Defining the classification task

For now, I define the same classification task as Guevara et al. (2024):

- **Task**: Multi-label sentence-level classification
- **SDoH**: 6 SDoH categories
    - Employment status
    - Housing issues
    - Transportation issues
    - Parental status
    - Relationship status
    - Social support

I define two levels of classification:

1. Level 1: *Any SDoH mentions*. The presence of language describing an SDoH category as defined above, regardless of the attribute. 
2. Level 2: *Adverse SDoH mentions*. The presence or absence of language describing an SDoH category with an attribute that could create an additional social work or resource support need for patients:
    - Employment status: unemployed, underemployed, disability
    - Housing issue: financial status, undomiciled, other
    - Transportation issue: distance, resources, other
    - Parental status: having a child under 18 years old
    - Relationship: widowed, divorced, single
    - Social support: absence of social support

In [8]:
import torch
from transformers import AutoTokenizer, AutoModel
import json
import re
from typing import List, Dict

from utils.model_helpers import *
from utils.classification_helpers import *

# Load one of the instruction-tuned models
# Qwen/Qwen2.5-7B-Instruct - Excellent for zero-shot classification
# meta-llama/Llama-3.1-8B-Instruct - Very capable instruction-following model
# microsoft/Phi-4-mini-instruct - Smaller but efficient
# mistralai/Mistral-7B-Instruct-v0.3 - Great performance on classification tasks

In [9]:
model_name = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer, model = None, None

print(f"Trying to load {model_name}...")
tokenizer, model = load_instruction_model(model_name)

Trying to load meta-llama/Llama-3.1-8B-Instruct...
Loading meta-llama/Llama-3.1-8B-Instruct...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✓ meta-llama/Llama-3.1-8B-Instruct loaded successfully!


In [10]:
print(f'{sample_note}')

Befriending. Lives with husband for whom patient is carer. Living on ready meals at present. PERSON concerned that they may not be eating / drinking enough. Carers in XXXX times daily for patient to help with washing / dressing. Depending on side - effects of radiotherapy , patient may go on to PEG feeding. Patient feeling slightly overwhelmed by everything. FPOC and Carers Support Shropshire numbers given to patient ’s daughter ( with patient consent ) but would value additional emotional support / befriending / check ins to make sure they are eating / drinking etc as all activities of daily living severely compromised at present. Very supportive daughter who lives in PERSON. The patient is due to start radiotherapy on XXXX January so support during this time would be particularly beneficial. Due to start radiotherapy on XXXX at SATH EOS XXXX.


In [15]:
# Extract SDoH factors
results_level1 = extract_sdoh_from_note(
    sample_note, 
    model, 
    tokenizer, 
    prompt_type="five_shot_detailed", 
    level=1,
    debug=True,
)

print(json.dumps(results_level1, indent=2))


{
  "original_note": "Befriending. Lives with husband for whom patient is carer. Living on ready meals at present. PERSON concerned that they may not be eating / drinking enough. Carers in XXXX times daily for patient to help with washing / dressing. Depending on side - effects of radiotherapy , patient may go on to PEG feeding. Patient feeling slightly overwhelmed by everything. FPOC and Carers Support Shropshire numbers given to patient \u2019s daughter ( with patient consent ) but would value additional emotional support / befriending / check ins to make sure they are eating / drinking etc as all activities of daily living severely compromised at present. Very supportive daughter who lives in PERSON. The patient is due to start radiotherapy on XXXX January so support during this time would be particularly beneficial. Due to start radiotherapy on XXXX at SATH EOS XXXX.",
  "sentences": [
    {
      "sentence_number": 1,
      "sentence": "Lives with husband for whom patient is carer

In [ ]:
# Extract SDoH factors
results_level2 = extract_sdoh_from_note(
    sample_note, 
    model, 
    tokenizer, 
    prompt_type="five_shot_detailed", 
    level=2
)

print(json.dumps(results_level2, indent=2))

{
  "original_note": "Befriending. Lives with husband for whom patient is carer. Living on ready meals at present. PERSON concerned that they may not be eating / drinking enough. Carers in XXXX times daily for patient to help with washing / dressing. Depending on side - effects of radiotherapy , patient may go on to PEG feeding. Patient feeling slightly overwhelmed by everything. FPOC and Carers Support Shropshire numbers given to patient \u2019s daughter ( with patient consent ) but would value additional emotional support / befriending / check ins to make sure they are eating / drinking etc as all activities of daily living severely compromised at present. Very supportive daughter who lives in PERSON. The patient is due to start radiotherapy on XXXX January so support during this time would be particularly beneficial. Due to start radiotherapy on XXXX at SATH EOS XXXX.",
  "sentences": [
    {
      "sentence_number": 1,
      "sentence": "Lives with husband for whom patient is carer

## Batch extraction